# Coursera Capstone Project

In [1]:
# Load all the required Libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# !pip install bs4
from bs4 import BeautifulSoup

In [2]:
# Create a request to get the html from the webage then create and instance of beatifulsoup
webpage_request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
webpage = webpage_request.content
soup = BeautifulSoup(webpage, 'html.parser')

In [3]:
# Create a list to save all the scraped data from the table in 
table_contents=[]

# use the soup object to find the table in the html file
table=soup.find('table')

In [4]:
# loop through the table and append the needed informations into the table_contents list
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
# Create a pandas dataframe and save the list into it. 
df=pd.DataFrame(table_contents)

In [6]:
# clean the Borough column
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [19]:
# load the Geospatial_Coordinates csv file from the IBM database into a dataframe
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_ae3fa849039a4f62b3fd7d2348200875 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_ae3fa849039a4f62b3fd7d2348200875 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_ae3fa849039a4f62b3fd7d2348200875 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NlDZ-hXn1lZdMSqtqPdqPPhRqnjx3oT602jp1fmJd0AB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_ae3fa849039a4f62b3fd7d2348200875)

body = client_ae3fa849039a4f62b3fd7d2348200875.get_object(Bucket='courseracapstone-donotdelete-pr-rgsmcsg1cvgufi',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo_df = pd.read_csv(body)
geo_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
# merge Geospatial_Coordinates data with the neigborhood data 
merged_df = df.merge(geo_df, on='Postal Code')
merged_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
